# SVM Kernel Approximation

1. Environment Setup: import required libraries
2. Loading Dataset and Preprocessing Data
3. Build Input Fn
4. Training Model
5. Evaluating Training Data


## 1. Environment Setup: import required library

We include the required libraries that will be used in the next parts. The **time**, **numpy**, and **tensorflow** are common libraries in machine learning. The **fio**, which is "file input output" used to load data and **config**, which is "configuration file" used to config the path of the dataset files are written by myself. Modify it when you need it.

**patch/metrics.py**: fix `tf.metrics.true_negatives` method is missing on Tensorflow r1.4.

In [1]:
import fio
import preprocessing as pc
from config import *
import utility

# python std library
import gc
import time
import logging
import functools
import collections
from multiprocessing.pool import ThreadPool

# install library
import numpy as np
import tensorflow as tf

# patch
import patch.metrics

/Users/akiicat/opt/anaconda3/envs/universe/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/akiicat/opt/anaconda3/envs/universe/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/akiicat/opt/anaconda3/envs/universe/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/akiicat/opt/ana

## 2. Loading Dataset and Preprocessing Data

Loading the training set, validation set, and test set that was defined in **config.py** file. Sample files consisting of indices of data will be used to undersample the data.

There are two parts of the data that must be processed:

- the input data represented by the prefix "**X**" on variables
- the target data represented by the prefix "**Y**" on variables

Check out the file **preprocess.py** for more details.

In [2]:
def load(window_size):
    X_train = fio.load_file(X_train_dataset)
    Y_train = fio.load_file(Y_train_dataset)
    X_test = fio.load_file(X_test_dataset)
    Y_test = fio.load_file(Y_test_dataset)
    train_sample = fio.load_sample_file(train_sample_dataset)
    valid_sample = fio.load_sample_file(valid_sample_dataset)

    stat = pc.get_feat_stat(X_train)
    
    X_train = pc.standardize(X_train, stat)
    X_test = pc.standardize(X_test, stat)
    
    X_train = pc.expand(X_train, window_size)
    X_test = pc.expand(X_test, window_size)
    
    Y_train = pc.classify(Y_train)
    Y_test = pc.classify(Y_test)
    
    return X_train, Y_train, X_test, Y_test, train_sample, valid_sample

In [3]:
if __name__ == "__main__":
    window_size = 23
    
    X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, train_sample, valid_sample = load(window_size)
    tf.estimator.RunConfig
    print("the length of training set:", len(X_train_orig))
    print("the length of testing set:", len(X_test_orig))
    print("the first row training data:", np.sum(X_train_orig[0][0][0]))
    print("the first row target value:", np.sum(Y_train_orig[0][0][0]))

the length of training set: 3
the length of testing set: 2
the first row training data: 0.60705996
the first row target value: 1


## 3. Build Input Fn

- [Tensorflow Doc: dataset.from_generator](https://github.com/tensorflow/docs/blob/r1.4/site/en/api_docs/api_docs/python/tf/data/Dataset.md#from_generator)
- [Tensorflow Doc: dataset.batch](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#batch)
    - batch_size: A tf.int64 scalar tf.Tensor, representing the number of consecutive elements of this dataset to combine in a single batch.
    - drop_remainder: (Optional.) A tf.bool scalar tf.Tensor, representing whether the last batch should be dropped in the case it has fewer than batch_size elements; the default behavior is not to drop the smaller batch.
- [Tensorflow Doc: dataset.padded_batch](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#padded_batch)
- [How to use dataset in tensorflow](https://towardsdatascience.com/how-to-use-dataset-in-tensorflow-c758ef9e4428)
- [StackOverflow: Train Tensorflow model with estimator (from_generator)](https://stackoverflow.com/questions/49673602/train-tensorflow-model-with-estimator-from-generator?rq=1)
- [StackOverflow: Is Tensorflow Dataset API slower than Queues?](https://stackoverflow.com/questions/47403407/is-tensorflow-dataset-api-slower-than-queues)
- [Github: How can I ues Dataset to shuffle a large whole dataset?](https://github.com/tensorflow/tensorflow/issues/14857)

In [4]:
def np_input_fn(X, Y, samples=[], shuffle=False, window_size=1, batch=None, epoch=None):
    if batch is None:
        raise Exception('batch can not be None')
    
    if window_size & 0x1 == 0:
        raise Exception('window size can not even')
    dim_in = window_size * window_size * 6
    
    if not isinstance(X, list):
        X = [X]
    if not isinstance(Y, list):
        Y = [Y]
    if not isinstance(samples, list):
        samples = [samples]
    
    if len(samples) == 0:
        samples = [np.indices((x.shape[0], x.shape[1])).reshape((2,-1)).T for x in X]
    
    samples = [np.pad(x, ((0,0),(1,0)), 'constant', constant_values=i) for i, x in enumerate(samples)]
    samples = np.concatenate(samples)
    
    if shuffle == True:
        np.random.shuffle(samples)
    
    def generator():
        for s in samples:
            x = X[s[0]][s[1], s[2]].reshape((dim_in))
            y = Y[s[0]][s[1], s[2]].reshape((1))
            yield x, y
    
    def _input_fn():
        dataset = tf.data.Dataset.from_generator(generator,
                                                   output_types= (tf.float32, tf.int32), 
                                                   output_shapes=(tf.TensorShape([dim_in]), tf.TensorShape([1])))
        dataset = dataset.batch(batch_size=batch)
        dataset = dataset.repeat(epoch)
        dataset = dataset.prefetch(1)

        iterator = dataset.make_one_shot_iterator()
        features_tensors, labels = iterator.get_next()
        features = {'data': features_tensors }
        return features, labels
    
    return _input_fn


In [5]:
if __name__ == "__main__":
    print(np_input_fn(X_train_orig, Y_train_orig, train_sample, window_size=23, batch=128)())

({'data': <tf.Tensor 'IteratorGetNext:0' shape=(?, 3174) dtype=float32>}, <tf.Tensor 'IteratorGetNext:1' shape=(?, 1) dtype=int32>)


## 4. Training Model

The model that we used is followed by the article: [Improving Linear Models Using Explicit Kernel Methods](https://github.com/Debian/tensorflow/blob/master/tensorflow/contrib/kernel_methods/g3doc/tutorial.md).

[TensorFlow Estimators: Managing Simplicity vs. Flexibility in
High-Level Machine Learning Frameworks](https://storage.googleapis.com/pub-tools-public-publication-data/pdf/18d86099a350df93f2bd88587c0ec6d118cc98e7.pdf)

Optimizer

- [Ftrl Optimizer](https://www.tensorflow.org/api_docs/python/tf/compat/v1/train/FtrlOptimizer)
- [Adam Optimizer](https://www.tensorflow.org/api_docs/python/tf/compat/v1/train/AdamOptimizer)

Build the following models:

1. Build Linear Classifier Model
2. Build Random Fourier Feature Mapper Model and Linear Classifier Model

### 4.1. Training Linear Classifier Model

In [6]:
def create_linear_model(learning_rate, dim_in, config=None):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    image_column = tf.contrib.layers.real_valued_column('data', dimension=dim_in)
    
    estimator = tf.contrib.learn.LinearClassifier(
        feature_columns=[image_column],
        n_classes=2, 
        config=config,
        optimizer=optimizer)
    tf.estimator.train_and_evaluate
    return estimator

In [7]:
if __name__ == "__main__":
    batch = 128
    epoch = 2
    train_input_fn = np_input_fn(X_train_orig, Y_train_orig, train_sample, shuffle=True, window_size=23, batch=batch, epoch=epoch)
    
    learning_rate = 0.001       # Adam Optimizer
    input_dim = 23 * 23 * 6     # Data size
    
    estimator = create_linear_model(learning_rate, input_dim)

    start = time.time()
    estimator.fit(input_fn=train_input_fn) # Train.
    end = time.time()
    print('Elapsed time: {} seconds'.format(end - start))
    

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x61f446908>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/var/folders/40/1jhkx0kj6ld3fv5st6wpccsr0000gp/T/tmpifljajex'}
Instructions for updating:
Please switch to tf.train.get_global_step
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/40/1jhkx0kj6ld3fv5st6wpccsr0000gp/T/tmpifljajex/model.ckpt.
INFO:tensorflow:loss = 0.6931475, step = 1
INFO:

### 4.2. Training Random Fourier Feature Mapper Model

In [8]:
def create_rffm_model(learning_rate, dim_in, dim_out, stddev, config=None):
    
    kernel_mapper = tf.contrib.kernel_methods.RandomFourierFeatureMapper(dim_in, dim_out, stddev, name='rffm')
    
    optimizer = tf.train.AdamOptimizer(learning_rate)
    image_column = tf.contrib.layers.real_valued_column('data', dimension=dim_in)

    estimator = tf.contrib.kernel_methods.KernelLinearClassifier(
        feature_columns=[image_column], 
        n_classes=2, 
        config=config,
        optimizer=optimizer, 
        kernel_mappers={image_column: [kernel_mapper]})
    
    return estimator

In [9]:
if __name__ == "__main__":
    
    batch = 128
    epoch = 1
    train_input_fn = np_input_fn(X_train_orig, Y_train_orig, train_sample, shuffle=True, window_size=23, batch=batch, epoch=epoch)
    
    learning_rate = 0.001  # Adam Optimizer

    # RFFM
    input_dim = 23 * 23 * 6
    output_dim = 23 * 23 * 6 * 10
    stddev = 1.0

    estimator = create_rffm_model(learning_rate, input_dim, output_dim, stddev)
    
    start = time.time()
    estimator.fit(input_fn=train_input_fn) # Train.
    end = time.time()
    print('Elapsed time: {} seconds'.format(end - start))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x82a8d2c50>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/var/folders/40/1jhkx0kj6ld3fv5st6wpccsr0000gp/T/tmpjdr8eph_'}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/40/1jhkx0kj6ld3fv5st6wpccsr0000gp/T/tmpjdr8eph_/model.ckpt.
INFO:tensorflow:loss = 0.6931475, step = 1
INFO:tensorflow:global_step/sec: 1.32582
INFO:tensorflow:loss = 0.7007334,

## 5. Evaluating Training Data

1. Evaluating Training Data
2. Evaluating Validation Data
3. Evaluating Testing Data

**Confusion Matrix**

- [Classification: True vs. False and Positive vs. Negative](https://developers.google.com/machine-learning/crash-course/classification/true-false-positive-negative)
- [如何辨別機器學習模型的好壞？秒懂Confusion Matrix](https://www.ycc.idv.tw/confusion-matrix.html)

**estimator.evaluate**

- [Tensorflow Doc: estimator evaluate metrics](https://tensorflow.google.cn/versions/r1.15/api_docs/python/tf/keras/metrics)
- [Stack Overflow: Meaning of evaluation metrics in Tensorflow](https://ai.stackexchange.com/questions/6383/meaning-of-evaluation-metrics-in-tensorflow)

```python
x, y = {'data': X}, Y
input_fn = tf.estimator.inputs.numpy_input_fn(x, y, batch_size=batch, shuffle=False, num_epochs=1)
metric = estimator.evaluate(input_fn=input_fn)
```

**estimator.predict_classes**

```python
x, y = {'data': X_train.astype(np.float32) }, Y_train
batch = 128

input_fn = tf.estimator.inputs.numpy_input_fn(x, batch_size=batch, shuffle=False, num_epochs=1)
metric = estimator.predict_classes(input_fn=input_fn)

for i, p in enumerate(metric):
    print(p, y[i][0])
```

**Metrics**

- [Python tensorflow.contrib.learn.MetricSpec() Examples](https://www.programcreek.com/python/example/96156/tensorflow.contrib.learn.MetricSpec)
- [Tensorflow Doc: Available Metrics](https://github.com/tensorflow/docs/tree/r1.4/site/en/api_docs/api_docs/python/tf/metrics)

```python
metrics = { "accuracy": learn.MetricSpec(metric_fn=tf.metrics.accuracy, prediction_key="classes") }
metric = estimator.evaluate(input_fn=eval_input_fn, metrics=metrics)
```


In [10]:
def evaluate_model(estimator, X, Y, samples=[], window_size=1, batch=2048, epoch=1):

    eval_input_fn = np_input_fn(X, Y, samples, shuffle=False, window_size=window_size, batch=batch, epoch=epoch)
    
    metrics = {
        "tp": tf.contrib.learn.MetricSpec(metric_fn=tf.metrics.true_positives, prediction_key="classes"),
        "tn": tf.contrib.learn.MetricSpec(metric_fn=patch.metrics.true_negatives, prediction_key="classes"),
        "fp": tf.contrib.learn.MetricSpec(metric_fn=tf.metrics.false_positives, prediction_key="classes"),
        "fn": tf.contrib.learn.MetricSpec(metric_fn=tf.metrics.false_negatives, prediction_key="classes"),
    }
    
    start = time.time()
    metric = estimator.evaluate(input_fn=eval_input_fn, metrics=metrics)
    end = time.time()
    print('Elapsed time: {} seconds'.format(end - start))
    
    return metric


### 5.1 Evaluating Training Data

In [11]:
if __name__ == "__main__":
    start = time.time()
    metrics = evaluate_model(estimator, X_train_orig, Y_train_orig, samples=train_sample, window_size=23)
    end = time.time()
    print('Elapsed time: {} seconds'.format(end - start))
    print("training metrics:", metrics)


INFO:tensorflow:Starting evaluation at 2020-02-26-07:01:02
INFO:tensorflow:Restoring parameters from /var/folders/40/1jhkx0kj6ld3fv5st6wpccsr0000gp/T/tmpjdr8eph_/model.ckpt-126
INFO:tensorflow:Finished evaluation at 2020-02-26-07:02:29
INFO:tensorflow:Saving dict for global step 126: accuracy = 0.6796545, accuracy/baseline_label_mean = 0.5036664, accuracy/threshold_0.500000_mean = 0.6796545, auc = 0.7508, auc_precision_recall = 0.7478306, fn = 2556.0, fp = 2599.0, global_step = 126, labels/actual_label_mean = 0.5036664, labels/prediction_mean = 0.50204736, loss = 0.6164965, precision/positive_threshold_0.500000_mean = 0.68102604, recall/positive_threshold_0.500000_mean = 0.6846391, tn = 5388.0, tp = 5549.0
Elapsed time: 101.04163312911987 seconds
Elapsed time: 101.06525111198425 seconds
training metrics: {'loss': 0.6164965, 'accuracy': 0.6796545, 'labels/prediction_mean': 0.50204736, 'labels/actual_label_mean': 0.5036664, 'accuracy/baseline_label_mean': 0.5036664, 'auc': 0.7508, 'auc_p

### 5.2 Evaluating Validation Data

In [12]:
if __name__ == "__main__":
    start = time.time()
    metrics = evaluate_model(estimator, X_train_orig, Y_train_orig, samples=valid_sample, window_size=23)
    end = time.time()
    print('Elapsed time: {} seconds'.format(end - start))
    print("validation metrics:", metrics)


INFO:tensorflow:Starting evaluation at 2020-02-26-07:02:43
INFO:tensorflow:Restoring parameters from /var/folders/40/1jhkx0kj6ld3fv5st6wpccsr0000gp/T/tmpjdr8eph_/model.ckpt-126
INFO:tensorflow:Finished evaluation at 2020-02-26-07:03:19
INFO:tensorflow:Saving dict for global step 126: accuracy = 0.50465643, accuracy/baseline_label_mean = 0.49836394, accuracy/threshold_0.500000_mean = 0.50465643, auc = 0.4983432, auc_precision_recall = 0.490726, fn = 964.0, fp = 1004.0, global_step = 126, labels/actual_label_mean = 0.49836394, labels/prediction_mean = 0.5012047, loss = 0.72270024, precision/positive_threshold_0.500000_mean = 0.5029703, recall/positive_threshold_0.500000_mean = 0.5131313, tn = 989.0, tp = 1016.0
Elapsed time: 41.73852801322937 seconds
Elapsed time: 41.74473190307617 seconds
validation metrics: {'loss': 0.72270024, 'accuracy': 0.50465643, 'labels/prediction_mean': 0.5012047, 'labels/actual_label_mean': 0.49836394, 'accuracy/baseline_label_mean': 0.49836394, 'auc': 0.498343

### 5.3 Evaluating Testing Data

In [13]:
if __name__ == "__main__":
    
    print(X_test_orig[0].shape)
    start = time.time()
    metrics = evaluate_model(estimator, X_test_orig, Y_test_orig, window_size=23)
    end = time.time()
    print('Elapsed time: {} seconds'.format(end - start))
    print("testing metrics:", metrics)


(4, 3, 23, 23, 6)
INFO:tensorflow:Starting evaluation at 2020-02-26-07:03:25
INFO:tensorflow:Restoring parameters from /var/folders/40/1jhkx0kj6ld3fv5st6wpccsr0000gp/T/tmpjdr8eph_/model.ckpt-126
INFO:tensorflow:Finished evaluation at 2020-02-26-07:03:40
INFO:tensorflow:Saving dict for global step 126: accuracy = 0.495283, accuracy/baseline_label_mean = 0.509434, accuracy/threshold_0.500000_mean = 0.495283, auc = 0.50409544, auc_precision_recall = 0.5226319, fn = 48.0, fp = 59.0, global_step = 126, labels/actual_label_mean = 0.509434, labels/prediction_mean = 0.5050454, loss = 0.7004536, precision/positive_threshold_0.500000_mean = 0.5042017, recall/positive_threshold_0.500000_mean = 0.5555556, tn = 45.0, tp = 60.0
Elapsed time: 20.567481994628906 seconds
Elapsed time: 20.572479963302612 seconds
testing metrics: {'loss': 0.7004536, 'accuracy': 0.495283, 'labels/prediction_mean': 0.5050454, 'labels/actual_label_mean': 0.509434, 'accuracy/baseline_label_mean': 0.509434, 'auc': 0.50409544,